# Gamut Mapping - Medicina 01

## Introduction

This notebook implements an interactive widget used to investigate various gamut models in the context of the [ACES VWG on Gamut Mapping](https://community.acescentral.com/c/aces-development-acesnext/vwg-aces-gamut-mapping-working-group/).

It is assumed that the reader has basic knowledge about image processing and colour management in the Media and Entertainment Industry. An introductory publication on the topic is [Cinematic Color: From Your Monitor to the Big Screen](https://cinematiccolor.org/) by Selan (2012).

Images courtesy of:

- [Justin Holt](https://www.dropbox.com/sh/u6z2a0jboo4vno8/AAB-10qcflhpr0C5LWhs7Kq4a?dl=0)
- [Thomas Mansencal](https://community.acescentral.com/t/spectral-images-generation-and-processing/)
- [Fabian Matas](https://www.dropbox.com/sh/u6z2a0jboo4vno8/AAB-10qcflhpr0C5LWhs7Kq4a?dl=0)
- [Carol Payne](https://www.dropbox.com/sh/u6z2a0jboo4vno8/AAB-10qcflhpr0C5LWhs7Kq4a?dl=0)
- [Martin Smekal](https://community.acescentral.com/t/vfx-work-in-acescg-with-out-of-gamut-devices/2385)

## Imports & Overall Settings

In [1]:
%matplotlib widget

In [2]:
from __future__ import division, unicode_literals

import colour
import ipympl.backend_nbagg
import ipywidgets
import matplotlib.gridspec as gridspec
import matplotlib.patches as patches
import matplotlib.pyplot as plt
import numpy as np
import os
import scipy.interpolate
from matplotlib.collections import PathCollection
from matplotlib._layoutbox import plot_children

DEFAULT_BOX_DECORATION_WIDTH = 4
MPL_BOX_DECORATION_WIDTH = 28

COLOUR_STYLE = colour.plotting.colour_style()
COLOUR_STYLE.update({
    'legend.framealpha':
    colour.plotting.COLOUR_STYLE_CONSTANTS.opacity.low
})

plt.style.use(COLOUR_STYLE)

plt.style.use('dark_background')

colour.utilities.describe_environment()

colour.utilities.filter_warnings(*[True] * 4);

*                                                                             *
*   Interpreter :                                                             *
*       python : 3.7.6 (default, Dec 30 2019, 19:38:26)                       *
*                [Clang 11.0.0 (clang-1100.0.33.16)]                          *
*                                                                             *
*   colour-science.org :                                                      *
*       colour : v0.3.15-141-g3bebd7e9                                        *
*                                                                             *
*   Runtime :                                                                 *
*       imageio : 2.8.0                                                       *
*       matplotlib : 3.0.3                                                    *
*       numpy : 1.18.4                                                        *
*       scipy : 1.4.1                   

## Widgets Styling

CSS style for various widgets to improve overall presentation.

In [3]:
%%html
<style>
.widget-button {
    margin-left: 10px;
    margin-right: 10px;
}
</style>

## Colour Wheel Generation

Utility function responsible to produce a scene-referred colour wheel.

In [4]:
def colour_wheel(samples=1024, clip_circle=False, method='Colour'):
    xx, yy = np.meshgrid(
        np.linspace(-1, 1, samples), np.linspace(-1, 1, samples))

    S = np.sqrt(xx ** 2 + yy ** 2)    
    H = (np.arctan2(xx, yy) + np.pi) / (np.pi * 2)

    HSV = colour.utilities.tstack([H, S, np.ones(H.shape)])
    RGB = colour.HSV_to_RGB(HSV)

    if clip_circle == True:
        RGB[S > 1] = 0
        A = np.where(S > 1, 0, 1)
    else:
        A = np.ones(S.shape)

    if method.lower()== 'matplotlib':
        RGB = colour.utilities.orient(RGB, '90 CW')
    elif method.lower()== 'nuke':
        RGB = colour.utilities.orient(RGB, 'Flip')
        RGB = colour.utilities.orient(RGB, '90 CW')

    R, G, B = colour.utilities.tsplit(RGB)
    
    return colour.utilities.tstack([R, G, B, A])

## Utility Functions & Objects

Various random utility functions and objects.

In [5]:
def batch(sequence, count):
    for i in range(0, len(sequence), count):
        yield sequence[i:i + count]
        

def border_layout():
    return ipywidgets.widgets.Layout(
        border='solid {0}px #222'.format(4),
        margin='{0}px'.format(DEFAULT_BOX_DECORATION_WIDTH),
        padding='0')


def adjust_exposure(a, EV):
    a = colour.utilities.as_float_array(a)

    return a * pow(2, EV)

## View Transforms

Because the imagery represents scene-referred exposure values, the gamut mapped images should ideally be viewed with an appropriate *View Transform*, typically an *ACES RRT + sRGB ODT* transforms combination.

The notebook implements the following *View Transforms*:

- *sRGB*: Performs a transformation from the working space to the *sRGB* colourspace and applies the *sRGB* inverse *Electro-Optical Transfer Function*.
- *sRGB (ACES)*: Performs a transformation from the working space to the *[ACES sRGB Output Device Transform](https://github.com/ampas/aces-dev/blob/master/transforms/ctl/odt/sRGB/ODT.Academy.sRGB_100nits_dim.ctl)*.

In [6]:
def view_transform_sRGB(RGB, working_space):
    RGB = colour.RGB_to_RGB(RGB, working_space, colour.models.sRGB_COLOURSPACE,
                            apply_encoding_cctf=True)

    return np.clip(RGB, 0, 1)


def view_transform_Display_P3(RGB, working_space):
    RGB = colour.RGB_to_RGB(RGB, working_space, colour.models.DISPLAY_P3_COLOURSPACE,
                            apply_encoding_cctf=True)

    return np.clip(RGB, 0, 1)


ACES_sRGB_LUT = colour.read_LUT('../resources/luts/ACES2065-1_sRGB_OT.csp')
ACES_Display_P3_LUT = colour.read_LUT('../resources/luts/ACES2065-1_display_P3_OT.csp')


def view_transform_sRGB_ACES(RGB, working_space):
    RGB = colour.RGB_to_RGB(RGB, working_space,
                            colour.models.ACES_2065_1_COLOURSPACE)
    RGB = ACES_sRGB_LUT.apply(np.clip(RGB, 0.001186, 222.860992))

    return RGB


def view_transform_Display_P3_ACES(RGB, working_space):
    RGB = colour.RGB_to_RGB(RGB, working_space,
                            colour.models.ACES_2065_1_COLOURSPACE)
    RGB = ACES_Display_P3_LUT.apply(np.clip(RGB, 0.001186, 222.860992))

    return RGB


VIEW_TRANSFORMS = {
    'sRGB': view_transform_sRGB,
    'sRGB (ACES)': view_transform_sRGB_ACES,
    'Display P3': view_transform_Display_P3,
    'Display P3 (ACES)': view_transform_Display_P3_ACES,
}

## Gamut Medicina Base Widget

The `GamutMedicinaBaseWidget` base widget class is inherited by the concrete sub-classes implementing the various gamut mapping study models.

At its core it is itself an `ipywidgets.widgets.Box` sub-class which outputs a [Matplotlib](http://matplotlib.org/) figure composed of three sub-plots:

- **Reference Image**: Required, 1 cartesian `Axes`
- **Colour Wheel**: Optional, 1 cartesian `Axes` and 1 polar `Axes`
- **Chromaticity Diagram**: Optional, 1 cartesian `Axes`

Trivial layout heuristic is implemented to support optionally enabling or disabling the **Colour Wheel** and **Chromaticity Diagram** sub-plots.

### Reference Image

The **Reference Image** sub-plot is responsible for displaying the gamut mapped scene-referred imagery.

*Note*

> To improve the performance of the notebook, the *View Transform*, i.e. a S-Curve, converting the scene-referred imagery to output-referred values is omitted.

### Colour Wheel

The **Colour Wheel** sub-plot is used to represent the controls for the gamut mapping study models adopting a cylindrical or conic working space.

The colour wheel is drawn on a cartesian `Axes` on top of which is overlayed a polar `Axes` used to draw the control regions.

### Chromaticity Diagram

The **Chromaticity Diagram** sub-plot is also used to represent the controls for the gamut mapping study models into the *Chromaticity Diagram 1976 UCS* based on the [CIE L\*u\*v\*](https://fr.wikipedia.org/wiki/L*u*v*_CIE_1976) colourspace.

It also features the following elements:

- *RGB Working Space* gamut, i.e. *ACEScg/AP1*, large triangle
- *RGB Display Space* gamut, i.e. *sRGB*, small triangle
- Scatter of the decimated **Reference Image** data
- Scatter of the ColorChecker Classic data, white scatter points


In [7]:
class GamutMedicinaBaseWidget(ipywidgets.widgets.Box):
    def __init__(self,
                 reference_image_path=None,
                 working_space=colour.models.ACES_CG_COLOURSPACE,
                 reference_space=colour.models.ACES_2065_1_COLOURSPACE,
                 display_space=colour.models.sRGB_COLOURSPACE,
                 view_transform='sRGB',
                 colour_wheel_samples=256,
                 image_decimation=10,
                 reference_image_exposure=0,
                 figure_size=(11, None),
                 padding=0.005,
                 spacing=0.005,
                 show_labels=False,
                 enable_colour_wheel=True,
                 enable_chromaticity_diagram=True,
                 show_out_of_gamut=False,
                 exposure_value=0,
                 enable_medicina=True,
                 debug_layout=False):
        super().__init__()

        self._reference_image_path = reference_image_path
        self._working_space = working_space
        self._reference_space = reference_space
        self._display_space = display_space
        self._view_transform = view_transform
        self._colour_wheel_samples = colour_wheel_samples
        self._image_decimation = image_decimation
        self._reference_image_exposure = reference_image_exposure
        self._figure_size = figure_size
        self._padding = padding
        self._spacing = spacing
        self._show_labels = show_labels
        self._exposure_value = exposure_value
        self._enable_colour_wheel = enable_colour_wheel
        self._enable_chromaticity_diagram = enable_chromaticity_diagram

        self._show_out_of_gamut = show_out_of_gamut
        self._enable_medicina = enable_medicina

        self._reference_image = None
        self._reference_image_pre_working = None
        self._reference_image_working = None
        self._reference_image_mapped = None
        self._reference_image_display = None

        if self._enable_colour_wheel:
            self._colour_wheel = None

        if self._enable_chromaticity_diagram:
            self._decimated_image_pre_working = None
            self._decimated_image_working = None
            self._decimated_image_display = None

        self._colour_checker = None
        self._colour_checker_uv = None

        self._output = None
        self._figure = None
        self._grid_spec = None
        self._reference_image_axes = None
        self._reference_image_imshow = None

        if self._enable_colour_wheel:
            self._colour_wheel_cartersian_axes = None
            self._colour_wheel_imshow = None
            self._colour_wheel_polar_axes = None

        if self._enable_chromaticity_diagram:
            self._chromaticity_diagram_axes = None

        self._widget_name_HTML = None
        self._disable_medidicina_CheckBox = None
        self._export_reference_image_Button = None
        self._save_figure_image_Button = None
        self._view_transform_DropDown = None
        self._reset_exposure_Button = None
        self._exposure_FloatSlider = None
        self._show_out_of_gamut_CheckBox = None
        self._global_controls_HBox = None
        self._viewer_controls_HBox = None
        self._controls_Tab = None

        self.initialize_data()
        self.initialize_axes()
        self.initialise_widgets()
        self.attach_callbacks()

        if debug_layout:
            plot_children(self._figure, self._figure._layoutbox, printit=False)

    def initialize_data(self):
        # *** Reference Image ***
        self._reference_image = colour.utilities.as_float_array((
            colour.read_image(self._reference_image_path)
        ) if self._reference_image_path is not None else np.random.rand(
            540, 960, 3))
        self._reference_image_path = (self._reference_image_path
                                      if self._reference_image_path is not None
                                      else 'Random.exr')
        self._reference_image_pre_working = self.reference_space_to_working_space(
            self._reference_image)

        self._reference_image_working = self._reference_image_pre_working
        self._reference_image_mapped = self._reference_image_pre_working
        self._reference_image_display = self.working_space_to_display_space(
            self._reference_image_working)

        # *** Colour Wheel ***
        if self._enable_colour_wheel:
            self._colour_wheel = colour_wheel(
                self._colour_wheel_samples, method='Matplotlib')[..., 0:3]

        # *** Decimated Image, i.e. Scatter ***
        if self._enable_chromaticity_diagram:
            self._decimated_image_pre_working = (
                self._reference_image_working[::self._image_decimation, ::self.
                                              _image_decimation, :]).reshape(
                                                  -1, 3)

            self._decimated_image_pre_working = \
            self._decimated_image_pre_working[
                self._decimated_image_pre_working[:, 1].argsort()]
            self._decimated_image_working = self._decimated_image_pre_working
            self._decimated_image_display = (
                self.working_space_to_display_space(
                    self._decimated_image_working))

        # *** Colour Checker
        colour_checker = colour.COLOURCHECKERS[
            'ColorChecker24 - After November 2014']
        colour_checker_data = colour.utilities.as_float_array(
            list(colour_checker.data.values()))
        self._colour_checker_uv = colour.xy_to_Luv_uv(
            colour_checker_data[:, 0:2])

    def initialize_axes(self):
        self._output = ipywidgets.widgets.Output()

        image_height, image_width, channels = self._reference_image.shape
        if self._figure_size[-1] is None:
            if self._enable_colour_wheel and self._enable_chromaticity_diagram:
                width = image_height / 2 + image_width
                height = image_height - self._padding * 100 - self._spacing * 100
                # Unresolved fudge factor to ensure plots line up.
                height -= int(height * 0.05)
            elif self._enable_colour_wheel or self._enable_chromaticity_diagram:
                width = image_height + image_width
                height = image_height
            else:
                width = image_width
                height = image_height
            ratio = self._figure_size[0] * 100 / width
            figure_size = (width / 100 * ratio, height / 100 * ratio)
        else:
            figure_size = self._figure_size

        with self._output:
            self._figure = plt.figure(
                figsize=figure_size,
                constrained_layout=True,
                facecolor=[2 / 3 / 10] * 3)
            self._figure.canvas.toolbar_visible = False
            self._figure.canvas.header_visible = False
            self._figure.canvas.footer_visible = False
            self._figure.canvas.resizable = False
            self._figure.tight_layout()

        if self._enable_colour_wheel and self._enable_chromaticity_diagram:
            rows = columns = 2
            colour_wheel_indices = 0, 0
            chromaticity_diagram_indices = 1, 0
            reference_image_indices = slice(0, None, None), slice(
                1, None, None)
            width_ratios = [1, 2 * image_width / image_height]
        elif not self._enable_colour_wheel and not self._enable_chromaticity_diagram:
            rows = columns = 1
            colour_wheel_indices = chromaticity_diagram_indices = None
            reference_image_indices = 0, 0
            width_ratios = [1]
        else:
            rows = 1
            columns = 2
            colour_wheel_indices = chromaticity_diagram_indices = 0, 0
            reference_image_indices = 0, 1
            width_ratios = [1, image_width / image_height]

        self._grid_spec = gridspec.GridSpec(
            ncols=columns,
            nrows=rows,
            figure=self._figure,
            width_ratios=width_ratios,
            wspace=self._spacing,
            hspace=self._spacing)
        self._figure.set_constrained_layout_pads(
            w_pad=self._padding, h_pad=self._padding)

        # Colour Wheel Axes
        if self._enable_colour_wheel:
            self._colour_wheel_cartersian_axes = self._figure.add_subplot(
                self._grid_spec[colour_wheel_indices[0], colour_wheel_indices[
                    1]],
                label='Cartesian Axes')
            self._colour_wheel_cartersian_axes.axis('off')
            circle = patches.Circle(
                [0.5, 0.5],
                radius=0.5,
                transform=self._colour_wheel_cartersian_axes.transData)
            self._colour_wheel_imshow = self._colour_wheel_cartersian_axes.imshow(
                np.clip(self._colour_wheel, 0, 1),
                extent=[0, 1, 0, 1],
                clip_path=None)
            self._colour_wheel_imshow.set_clip_path(circle)

            self._colour_wheel_polar_axes = self._figure.add_subplot(
                self._grid_spec[colour_wheel_indices[0], colour_wheel_indices[
                    1]],
                projection='polar',
                label='Polar Axes')
            self._colour_wheel_polar_axes.set_xlim(0, np.pi * 2)
            self._colour_wheel_polar_axes.set_ylim(0, 1)
            self._colour_wheel_polar_axes.patch.set_alpha(0)
            self._colour_wheel_polar_axes.grid(alpha=0.15)
            if not self._show_labels:
                self._colour_wheel_polar_axes.set_xticklabels([])
            self._colour_wheel_polar_axes.set_yticklabels([])

        # Chromaticity Diagram Axes
        if self._enable_chromaticity_diagram:
            self._chromaticity_diagram_axes = self._figure.add_subplot(
                self._grid_spec[chromaticity_diagram_indices[0],
                                chromaticity_diagram_indices[1]])
            self._chromaticity_diagram_axes.patch.set_alpha(0)
            (colour.plotting.
             plot_RGB_chromaticities_in_chromaticity_diagram_CIE1976UCS(
                 self._decimated_image_working,
                 colourspace=self._working_space,
                 colourspaces=['sRGB'],
                 axes=self._chromaticity_diagram_axes,
                 standalone=False,
                 transparent_background=False,
                 spectral_locus_colours='RGB',
                 spectral_locus_labels=[],
                 diagram_opacity=0,
                 scatter_kwargs={'s': 1},
                 title=str(),
                 x_label=str(),
                 y_label=str(),
                 legend=False))
            self._chromaticity_diagram_axes.tick_params(
                axis='y', which='both', direction='in')
            self._chromaticity_diagram_axes.tick_params(
                axis='x', which='both', direction='in')
            self._chromaticity_diagram_axes.minorticks_off()
            self._chromaticity_diagram_axes.grid(alpha=0.15)
            for collection in self._chromaticity_diagram_axes.collections:
                if isinstance(collection, PathCollection):
                    self._scatter_path_collection = collection
                    break

            if not self._show_labels:
                self._chromaticity_diagram_axes.set_xticklabels([])
                self._chromaticity_diagram_axes.set_yticklabels([])

            self._chromaticity_diagram_axes.scatter(
                self._colour_checker_uv[:, 0],
                self._colour_checker_uv[:, 1],
                c='white',
                marker='D',
                s=5)

        # Reference Image Axes
        self._reference_image_axes = self._figure.add_subplot(self._grid_spec[
            reference_image_indices[0], reference_image_indices[1]])
        self._reference_image_axes.set_xticks([])
        self._reference_image_axes.set_yticks([])
        self._reference_image_imshow = self._reference_image_axes.imshow(
            self.working_space_to_display_space(self._reference_image_working))

    def initialise_widgets(self):
        # *** Widgets ***
        self._widget_name_HTML = ipywidgets.widgets.HTML(
            value=('<span style="color: #3498DB; font-size: 1.25rem">'
                   '<b>Gamut Medicina Widget</b></span>'))
        self._disable_medidicina_CheckBox = (ipywidgets.widgets.Checkbox(
            value=not self._enable_medicina, description='Disable Medicina'))
        self._export_reference_image_Button = (
            ipywidgets.widgets.Button(description="Export Image"))
        self._save_figure_image_Button = (
            ipywidgets.widgets.Button(description="Save Figure"))

        view_transforms = list(VIEW_TRANSFORMS.keys())
        self._view_transform_DropDown = ipywidgets.widgets.Dropdown(
            options=view_transforms,
            value=self._view_transform,
            description='View Transform:',
        )
        self._reset_exposure_Button = (
            ipywidgets.widgets.Button(description="Reset Exposure"))
        self._exposure_FloatSlider = (ipywidgets.widgets.FloatSlider(
            min=-8.0,
            max=8.0,
            step=0.1,
            value=self._reference_image_exposure,
            description='Exposure'))
        self._show_out_of_gamut_CheckBox = (ipywidgets.widgets.Checkbox(
            value=self._show_out_of_gamut, description='Show Out-of-Gamut'))

        # *** Layout ***
        self.layout.display = 'flex'
        self.layout.flex_flow = 'column'
        self.layout.align_items = 'stretch'
        self.layout.width = '{0}px'.format(self._figure_size[0] * 100 +
                                           MPL_BOX_DECORATION_WIDTH)

        self._global_controls_HBox = ipywidgets.widgets.HBox([
            self._widget_name_HTML,
            self._disable_medidicina_CheckBox,
            self._export_reference_image_Button,
            self._save_figure_image_Button,
        ])

        self._viewer_controls_HBox = ipywidgets.widgets.HBox([
            self._view_transform_DropDown,
            self._reset_exposure_Button,
            self._exposure_FloatSlider,
            self._show_out_of_gamut_CheckBox,
        ])

        self._controls_Tab = ipywidgets.widgets.Tab(children=[
            self._global_controls_HBox,
            self._viewer_controls_HBox,
        ])
        self._controls_Tab.set_title(0, 'Global Controls')
        self._controls_Tab.set_title(1, 'Viewer Controls')

        self._controls_Tab.layout = border_layout()

        self._output.layout = border_layout()
        self.children = [self._output, self._controls_Tab]

    def attach_callbacks(self):
        self._disable_medidicina_CheckBox.observe(self.toggle_medidicina)
        self._export_reference_image_Button.on_click(
            self.export_reference_image)
        self._save_figure_image_Button.on_click(self.save_figure)
        self._show_out_of_gamut_CheckBox.observe(self.toggle_out_of_gamut)
        self._reset_exposure_Button.on_click(self.reset_exposure)
        self._exposure_FloatSlider.observe(self.update_widget, 'value')
        self._view_transform_DropDown.observe(self.update_widget, 'value')

    def reference_space_to_working_space(self, RGB):
        return colour.RGB_to_RGB(RGB, self._reference_space,
                                 self._working_space)

    def working_space_to_reference_space(self, RGB):
        return colour.RGB_to_RGB(RGB, self._working_space,
                                 self._reference_space)

    def working_space_to_display_space(self, RGB, show_out_of_gamut=False):
        RGB = np.copy(RGB)

        if show_out_of_gamut:
            RGB[RGB > 0] = 0
            RGB[RGB < 0] = 1

        return VIEW_TRANSFORMS[self._view_transform](RGB, self._working_space)

    def update_scatter_plot(self, RGB):
        if self._enable_chromaticity_diagram:
            XYZ = colour.RGB_to_XYZ(RGB, self._working_space.whitepoint,
                                    self._working_space.whitepoint,
                                    self._working_space.RGB_to_XYZ_matrix)
            uv = colour.Luv_to_uv(
                colour.XYZ_to_Luv(XYZ, self._working_space.whitepoint),
                self._working_space.whitepoint)

            self._scatter_path_collection.set_offsets(uv)
            self._scatter_path_collection.set_facecolor(
                self.working_space_to_display_space(
                    RGB,
                    show_out_of_gamut=self._show_out_of_gamut_CheckBox.value))

    def toggle_medidicina(self, change):
        if not change:
            return

        if change['name'] == 'value':
            self._enable_medicina = not change['new']

            self.update_widget(None)

    def export_reference_image(self, change):
        path = os.path.splitext(self._reference_image_path)[0]

        colour.write_image(
            self.working_space_to_reference_space(
                self._reference_image_mapped), '{0}_{1}.exr'.format(
                    path, self.__class__.__name__))

    def save_figure(self, change):
        path = os.path.splitext(self._reference_image_path)[0]
        self._figure.savefig(
            '{0}_{1}.png'.format(path, self.__class__.__name__),
            facecolor=[2 / 3 / 10] * 3,
            transparent=False,
            bbox_inches='tight')

    def reset_exposure(self, change):
        self._exposure_FloatSlider.value = 0

        self.update_widget(None)

    def toggle_out_of_gamut(self, change):
        if not change:
            return

        if change['name'] == 'value':
            self.update_widget(None)

    def update_widget(self, change):
        self._view_transform = self._view_transform_DropDown.value

        if (not self._enable_medicina
                or self.__class__.__name__ == 'GamutMedicinaBaseWidget'):            
            self._reference_image_working = self._reference_image_pre_working
            self._decimated_image_working = self._decimated_image_pre_working

            EV = self._exposure_FloatSlider.value
            if EV != self._exposure_value:
                self._exposure_value = EV

                self._reference_image_working = adjust_exposure(
                    self._reference_image_pre_working, self._exposure_value)

                if self._enable_chromaticity_diagram:
                    self._decimated_image_working = adjust_exposure(
                        self._decimated_image_pre_working,
                        self._exposure_value)

            self._reference_image_imshow.set_data(
                self.working_space_to_display_space(
                    self._reference_image_working,
                    show_out_of_gamut=self._show_out_of_gamut_CheckBox.value))

            if self._enable_chromaticity_diagram:
                self.update_scatter_plot(self._decimated_image_working)

            self._figure.canvas.draw_idle()


GamutMedicinaBaseWidget()

GamutMedicinaBaseWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0')), T…

In [8]:
GamutMedicinaBaseWidget(enable_colour_wheel=False)

GamutMedicinaBaseWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0')), T…

In [9]:
GamutMedicinaBaseWidget(enable_chromaticity_diagram=False)

GamutMedicinaBaseWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0')), T…

In [10]:
GamutMedicinaBaseWidget(
    enable_colour_wheel=False, enable_chromaticity_diagram=False)

GamutMedicinaBaseWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0')), T…

## Compression & Blending Functions

Gamut mapping commonly involves compressing data from a larger space to a smaller space.

Three sigmoid compression functions are defined:

- **tanh**, a function based on the *Hyperbolic Tangent*:
$$a + b \cdot tanh\bigg(\cfrac{x - a}{b}\bigg)$$ where $a$ is the compression threshold, i.e. the point at which the function starts compressing the value $x$ and $b$ is the compression limit, i.e. the point at which the compression reaches the limit.
- **atan**, a function based on the *Arc Tangent*:
$$a + b \cdot\cfrac{2}{\pi}\cdot atan\Bigg(\cfrac{\cfrac{\pi}{2}\cdot\big(x - a\big)}{b}\Bigg)$$
- **Simple**, a simple compression function:
$$a + \Bigg(\cfrac{-1}{\bigg(\cfrac{x - a}{b} + 1\bigg)} + 1 \Bigg) \cdot b$$

In [11]:
def tanh_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, a + b * np.tanh((x - a) / b), x)


def atan_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, a + b * 2 / np.pi * np.arctan(((np.pi / 2) * (x - a)) / b), x)


def simple_compression_function(x, a=0.8, b=1 - 0.8):
    x = colour.utilities.as_float_array(x)

    return np.where(x > a, a + (-1 / ((x - a) / b + 1) + 1) * b, x)


COMPRESSION_FUNCTIONS = {
    'tanh' : tanh_compression_function,
    'atan' : atan_compression_function,
    'simple' : simple_compression_function,
}


def smoothstep(x, a=0, b=1):
    i = np.clip((x - a) / (b - a), 0, 1)

    return i**2 * (3 - 2 * i)


def derivative(x, func, d=1e-7):
    return (func(x + d) - func(x - d)) / (2 * d)

In [12]:
colour.plotting.plot_multi_functions(
    {
        'tanh': tanh_compression_function,
        'd/dx(tanh)': lambda x: derivative(x, tanh_compression_function),
        'atan': atan_compression_function,
        'd/dx(atan)': lambda x: derivative(x, atan_compression_function),
        'simple': simple_compression_function,
        'd/dx(simple)': lambda x: derivative(x, simple_compression_function),
    },
    **{
        'figure.figsize': (11, 11),
        'bounding_box': (0.5, 1.75, 0, 1.25),
        'samples':
        np.linspace(0, 2, 1000),
        'plot_kwargs': [
            {
                'c': 'r',
                'linestyle': 'dashdot'
            },
            {
                'c': 'r'
            },
            {
                'c': 'g',
                'linestyle': 'dashdot'
            },
            {
                'c': 'g'
            },
            {
                'c': 'b',
                'linestyle': 'dashdot'
            },
            {
                'c': 'b'
            },
        ]
    },
);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Mansencal and Scharfenberg (2020) - HSV Control Based Study Model

*Mansencal and Scharfenberg (2020)* gamut mapping study model is built on top of the [HSV colourspace](https://en.wikipedia.org/wiki/HSL_and_HSV):

- Scene-referred RGB exposure values are converted to HSV.
- The saturation component $S$ is compressed with a cherry-picked compression function, e.g. $tanh$.
- The hue component $H$ is warped according to user defined control values.
- The HSV values are converted back to scene-referred RGB exposure values and then blended with the original scene-referred RGB exposure values function through a smoothstep function.

The model offers good controls but might tend to exhibit excessive saturation of secondary colours, i.e. cyan, yellow and especially magenta.
This can be tuned with the hue controls.

More information is available on [ACEScentral](https://community.acescentral.com/t/gamut-mapping-in-cylindrical-and-conic-spaces/).

In [13]:
def medicina_HSV_control(RGB, H, S, V, H_x, H_y, S_c, S_m, C_f, HSV_to_RGB):
    S_m = S_m[..., np.newaxis]

    interpolator = scipy.interpolate.interp1d(H_x, H_y)

    HSV_c = colour.utilities.tstack(
        [interpolator(H) % 1, C_f(S, S_c, 1 - S_c), V])

    return HSV_to_RGB(HSV_c) * S_m + RGB * (1 - S_m)


class GamutMedicinaHSVControlWidget(GamutMedicinaBaseWidget):
    def __init__(self,
                 reference_image=None,
                 hue_wedge=45,
                 protected_area_threshold=0.7,
                 compression_threshold=0.8,
                 compression_function='tanh',
                 RGB_to_HSV=colour.RGB_to_HSV,
                 HSV_to_RGB=colour.HSV_to_RGB,
                 *args,
                 **kwargs):

        self._hue_wedge = hue_wedge
        self._protected_area_threshold = protected_area_threshold
        self._compression_threshold = compression_threshold
        self._compression_function = compression_function

        self.RGB_to_HSV = RGB_to_HSV
        self.HSV_to_RGB = HSV_to_RGB

        super().__init__(reference_image, *args, **kwargs)

        self._RGB_r = None
        self._H_r = self._S_r = self._V_r = None
        self._S_r_m = None

        if self._enable_colour_wheel:
            self._RGB_w = None
            self._H_w = self._S_w = self._V_w = None
            self._S_w_m = None

        if self._enable_chromaticity_diagram:
            self._RGB_d = None
            self._H_d = self._S_d = self._V_d = None
            self._S_d_m = None

        self._angles = None
        self._x_i = self._y_i = self._y_i = None

        if self._enable_colour_wheel:
            self._protected_area_threshold_colour_wheel_plot = None
            self._compression_threshold_colour_wheel_plot = None
            self._hue_controls_plot = None

        if self._enable_chromaticity_diagram:
            self._protected_area_threshold_chromaticity_diagram_plot = None
            self._compression_threshold_chromaticity_diagram_plot = None

        self._reset_protected_area_controls_Button = None
        self._protected_area_threshold_FloatSlider = None
        self._reset_compression_controls_Button = None
        self._compression_threshold_FloatSlider = None
        self._reset_hue_controls_Button = None
        self._wedge_controls = None
        self._protected_area_controls_HBox = None
        self._saturation_controls_HBox = None
        self._hue_controls_HBox = None

        self.initialize_data()
        self.initialize_axes()
        self.initialise_widgets()
        self.attach_callbacks()

        self.update_widget(None)

    def initialize_data(self):
        super().initialize_data()

        # *** Reference Image ***
        self._RGB_r = self._reference_image_working
        self._H_r, self._S_r, self._V_r = colour.utilities.tsplit(
            self.RGB_to_HSV(self._reference_image_working))
        self._S_r_m = smoothstep(self._S_r, self._protected_area_threshold,
                                 self._compression_threshold)

        # *** Colour Wheel ***
        if self._enable_colour_wheel:
            self._RGB_w = self._colour_wheel[..., 0:3]
            self._H_w, self._S_w, self._V_w = colour.utilities.tsplit(
                self.RGB_to_HSV(self._RGB_w))
            self._S_w_m = smoothstep(self._S_w, self._protected_area_threshold,
                                     self._compression_threshold)

        # *** Decimated Image, i.e. Scatter ***
        if self._enable_chromaticity_diagram:
            self._RGB_d = self._decimated_image_working
            self._H_d, self._S_d, self._V_d = colour.utilities.tsplit(
                self.RGB_to_HSV(self._decimated_image_working))
            self._S_d_m = smoothstep(self._S_d, self._protected_area_threshold,
                                     self._compression_threshold)

        # *** Angles ***
        self._angles = np.arange(0, 360, self._hue_wedge) / 360

        # *** Initial State ***
        self._x_i = np.hstack([self._angles, 1])
        self._y_i = self._angles
        self._y_i = np.hstack([self._y_i, self._y_i[0] + 1])

    def initialize_axes(self):
        super().initialize_axes()

        circumference = np.linspace(0, np.pi * 2, self._colour_wheel_samples)
        # Colour Wheel Axes
        if self._enable_colour_wheel:
            self._protected_area_threshold_colour_wheel_plot = (
                self._colour_wheel_polar_axes.plot(
                    circumference,
                    np.full(self._colour_wheel_samples,
                            self._protected_area_threshold),
                    linestyle='dotted',
                    color='yellow')[0])
            self._compression_threshold_colour_wheel_plot = (
                self._colour_wheel_polar_axes.plot(
                    circumference,
                    np.full(self._colour_wheel_samples,
                            self._compression_threshold),
                    linestyle='dashdot',
                    color='cyan')[0])

            self._hue_controls_plot = (self._colour_wheel_polar_axes.plot(
                self._y_i * np.pi * 2,
                np.ones(self._y_i.shape),
                'o-',
                color='white')[0])

        # Chromaticity Diagram Axes
        if self._enable_chromaticity_diagram:
            self._protected_area_threshold_chromaticity_diagram_plot = (
                self._chromaticity_diagram_axes.plot(
                    circumference * 0,
                    circumference * 0,
                    linestyle='dotted',
                    color='yellow')[0])
            self._compression_threshold_chromaticity_diagram_plot = (
                self._chromaticity_diagram_axes.plot(
                    circumference * 0,
                    circumference * 0,
                    linestyle='dashdot',
                    color='cyan')[0])

    def initialise_widgets(self):
        super().initialise_widgets()

        # *** Widgets ***
        self._widget_name_HTML.value = (
            '<span style="color: #3498DB; font-size: 1.25rem">'
            '<b>HSV Control Widget</b></span>')

        self._reset_protected_area_controls_Button = (
            ipywidgets.widgets.Button(description="Reset Protected Area"))
        self._protected_area_threshold_FloatSlider = (
            ipywidgets.widgets.FloatSlider(
                min=0.0,
                max=1.0,
                step=0.01,
                value=self._protected_area_threshold,
                description='Protected Area Threshold'))

        self._reset_compression_controls_Button = (
            ipywidgets.widgets.Button(description="Reset Saturation Controls"))
        compression_functions = list(COMPRESSION_FUNCTIONS.keys())
        self._compression_function_DropDown = ipywidgets.widgets.Dropdown(
            options=compression_functions,
            value=self._compression_function,
            description='Compression Function:',
        )
        self._compression_threshold_FloatSlider = (
            ipywidgets.widgets.FloatSlider(
                min=0.0,
                max=1.0,
                step=0.01,
                value=self._compression_threshold,
                description='Compression Threshold'))

        self._reset_hue_controls_Button = ipywidgets.widgets.Button(
            description="Reset Hue Controls")
        self._wedge_controls = [
            ipywidgets.widgets.FloatSlider(
                min=-1,
                max=1,
                step=0.01,
                value=0,
                description='{0}°'.format(int(angle * 360)))
            for angle in self._angles
        ]

        # *** Layout ***
        self._protected_area_controls_HBox = ipywidgets.widgets.HBox([
            self._reset_protected_area_controls_Button,
            self._protected_area_threshold_FloatSlider,
        ])

        self._saturation_controls_HBox = ipywidgets.widgets.HBox([
            self._reset_compression_controls_Button,
            self._compression_function_DropDown,
            self._compression_threshold_FloatSlider
        ])

        wedge_controls_batches = batch(self._wedge_controls, 3)
        self._hue_controls_HBox = ipywidgets.widgets.HBox(
            [ipywidgets.widgets.VBox([self._reset_hue_controls_Button])] + [
                ipywidgets.widgets.VBox(wedge_controls)
                for wedge_controls in wedge_controls_batches
            ])

        self._controls_Tab.children += (
            self._protected_area_controls_HBox,
            self._saturation_controls_HBox,
            self._hue_controls_HBox,
        )

        self._controls_Tab.set_title(2, 'Protected Area Controls')
        self._controls_Tab.set_title(3, 'Saturation Controls')
        self._controls_Tab.set_title(4, 'Hue Controls')

    def attach_callbacks(self):
        super().attach_callbacks()

        self._reset_protected_area_controls_Button.on_click(
            self.reset_protected_area_controls)
        self._protected_area_threshold_FloatSlider.observe(
            self.update_protected_area_mask, 'value')

        self._reset_compression_controls_Button.on_click(
            self.reset_compression_controls)
        self._compression_function_DropDown.observe(self.update_widget,
                                                    'value')
        self._compression_threshold_FloatSlider.observe(
            self.update_widget, 'value')

        self._reset_hue_controls_Button.on_click(self.reset_hue_controls)
        for slider in self._wedge_controls:
            slider.observe(self.update_widget, 'value')

    def update_region_colour_wheel_plot(self, V_r, region_colour_wheel_plot):
        if self._enable_colour_wheel:
            region_colour_wheel_plot.set_ydata(
                np.full(self._colour_wheel_samples, V_r))

    def update_protected_area_threshold_colour_wheel_plot(self, U_r):
        if self._enable_colour_wheel:
            self.update_region_colour_wheel_plot(
                U_r, self._protected_area_threshold_colour_wheel_plot)

    def update_compression_threshold_colour_wheel_plot(self, S_c):
        if self._enable_colour_wheel:
            self.update_region_colour_wheel_plot(
                S_c, self._compression_threshold_colour_wheel_plot)

    def update_region_chromaticity_diagram_plot(
            self, V_r, region_chromaticity_diagram_plot):
        if self._enable_chromaticity_diagram:
            HSV = colour.utilities.tstack([
                np.linspace(0, 1, self._colour_wheel_samples),
                np.full(self._colour_wheel_samples, V_r),
                np.ones(self._colour_wheel_samples)
            ])
            uv = colour.Luv_to_uv(
                colour.XYZ_to_Luv(
                    colour.RGB_to_XYZ(
                        self.HSV_to_RGB(HSV), self._working_space.whitepoint,
                        self._working_space.whitepoint,
                        self._working_space.RGB_to_XYZ_matrix),
                    self._working_space.whitepoint))

            region_chromaticity_diagram_plot.set_data(uv[:, 0], uv[:, 1])

    def update_protected_area_threshold_chromaticity_diagram_plot(self, U_r):
        if self._enable_chromaticity_diagram:
            self.update_region_chromaticity_diagram_plot(
                U_r, self._protected_area_threshold_chromaticity_diagram_plot)

    def update_compression_threshold_chromaticity_diagram_plot(self, S_c):
        if self._enable_chromaticity_diagram:
            self.update_region_chromaticity_diagram_plot(
                S_c, self._compression_threshold_chromaticity_diagram_plot)

    def update_protected_area_mask(self, change):
        if not change:
            return

        if change['name'] == 'value':
            U_r = self._protected_area_threshold_FloatSlider.value
            S_c = self._compression_threshold_FloatSlider.value

            self._S_d_m = smoothstep(self._S_d, U_r, S_c)

            if self._enable_colour_wheel:
                self._S_w_m = smoothstep(self._S_w, U_r, S_c)

            if self._enable_chromaticity_diagram:
                self._S_r_m = smoothstep(self._S_r, U_r, S_c)

        self.update_widget(None)

    def wedge_control_to_hue_offset(self, value):
        slider = self._wedge_controls[0]

        return colour.utilities.linear_conversion(
            value, [slider.min, slider.max],
            [-self._hue_wedge / 360, self._hue_wedge / 360])

    def reset_protected_area_controls(self, change):
        self._protected_area_threshold_FloatSlider.value = (
            self._protected_area_threshold)

        self.update_protected_area_mask(None)

    def reset_compression_controls(self, change):
        self._compression_threshold_FloatSlider.value = (
            self._compression_threshold)

        self.update_widget(None)

    def reset_hue_controls(self, change):
        for slider in self._wedge_controls:
            slider.value = 0

        self.update_widget(None)

    def update_reference_image_axes(self, S_c, H_y, C_f, show_out_of_gamut):
        RGB_r = medicina_HSV_control(self._RGB_r, self._H_r, self._S_r,
                                     self._V_r, self._x_i, H_y, S_c,
                                     self._S_r_m, C_f, self.HSV_to_RGB)
        self._reference_image_mapped = RGB_r
        self._reference_image_imshow.set_data(
            self.working_space_to_display_space(
                RGB_r, show_out_of_gamut=show_out_of_gamut))

    def update_colour_wheel_axes(self, U_r, S_c, H_y, H_o, C_f):
        if self._enable_colour_wheel:
            RGB_w = medicina_HSV_control(self._RGB_w, self._H_w, self._S_w,
                                         self._V_w, self._x_i, H_y, S_c,
                                         self._S_w_m, C_f, self.HSV_to_RGB)
            self._colour_wheel_imshow.set_data(np.clip(RGB_w, 0, 1))
            self.update_protected_area_threshold_colour_wheel_plot(U_r)
            self.update_compression_threshold_colour_wheel_plot(S_c)

            H_x = H_o + self._angles
            H_x = np.hstack([H_x, H_x[0]])
            self._hue_controls_plot.set_xdata(H_x * np.pi * 2)

    def update_chromaticity_diagram_axes(self, U_r, S_c, H_y, C_f):
        if self._enable_chromaticity_diagram:
            RGB_r_s = medicina_HSV_control(self._RGB_d, self._H_d, self._S_d,
                                           self._V_d, self._x_i, H_y, S_c,
                                           self._S_d_m, C_f, self.HSV_to_RGB)
            self.update_scatter_plot(RGB_r_s)
            self.update_protected_area_threshold_chromaticity_diagram_plot(U_r)
            self.update_compression_threshold_chromaticity_diagram_plot(S_c)

    def update_widget(self, change):
        super().update_widget(change)

        if self._enable_medicina:
            EV = self._exposure_FloatSlider.value
            U_r = self._protected_area_threshold_FloatSlider.value
            S_c = self._compression_threshold_FloatSlider.value
            C_f = COMPRESSION_FUNCTIONS[
                self._compression_function_DropDown.value]
            show_out_of_gamut = self._show_out_of_gamut_CheckBox.value

            if EV != self._exposure_value:
                self._exposure_value = EV

                self._RGB_r = self._reference_image_working = adjust_exposure(
                    self._reference_image_pre_working,
                    EV)
                self._H_r, self._S_r, self._V_r = colour.utilities.tsplit(
                    self.RGB_to_HSV(self._RGB_r))

                if self._enable_chromaticity_diagram:
                    self._RGB_d = self._decimated_image_working = adjust_exposure(
                        self._decimated_image_pre_working, EV)
                    self._H_d, self._S_d, self._V_d = colour.utilities.tsplit(
                        self.RGB_to_HSV(self._RGB_d))

                self.update_protected_area_mask({'name': 'value'})

            hue_offsets = np.array([
                self.wedge_control_to_hue_offset(slider.value)
                for slider in self._wedge_controls
            ])

            H_y = -hue_offsets + self._angles
            H_y = np.hstack([H_y, H_y[0] + 1])

            self.update_reference_image_axes(S_c, H_y, C_f, show_out_of_gamut)

            if self._enable_colour_wheel:
                self.update_colour_wheel_axes(U_r, S_c, H_y, hue_offsets, C_f)

            if self._enable_chromaticity_diagram:
                self.update_chromaticity_diagram_axes(U_r, S_c, H_y, C_f)

            self._figure.canvas.draw_idle()


GamutMedicinaHSVControlWidget(
    '../resources/images/A009C002_190210_R0EI_Alexa_LogCWideGamut.exr')

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…

In [14]:
GamutMedicinaHSVControlWidget('../resources/images/A002_C029_06025K.exr')

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…

In [15]:
GamutMedicinaHSVControlWidget(
    '../resources/images/Cornell_Box_Rigid_Spheres_190_Patch_Roughplastic_Point_Grey_Grasshopper_50S5C_RGB_W_A.exr')

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…

In [16]:
GamutMedicinaHSVControlWidget(
    '../resources/images/5DMarkII_Spac-o-ween_001.exr',
    enable_colour_wheel=False)

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…

In [17]:
GamutMedicinaHSVControlWidget(
    '../resources/images/Collage_01.exr',
    enable_colour_wheel=False)

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…

## Smith (2020) - RGB Saturation Study Model

*Smith (2020)* gamut mapping study model is directly operating in the [RGB colourspace](https://en.wikipedia.org/wiki/Color_spaces_with_RGB_primaries):

- Like with the HSV colourspace, an achromatic axis is computed for the scene-referred RGB exposure values.
- The pseudo-distance between the individual $R$, $G$ and $B$ components and the achromatic axis is compressed with a cherry-picked compression function, e.g. $tanh$.

The model is extremely simple and elegant while offering good computational speed.

More information is available on [ACEScentral](https://community.acescentral.com/t/rgb-saturation-gamut-mapping-approach-and-a-comp-vfx-perspective/).

In [18]:
def medicina_RGB_saturation(RGB, C_t, C_f):
    C_t = 1 - C_t

    L = np.max(RGB, axis=-1)[..., np.newaxis]

    D = np.abs(RGB - L) / L

    D_c = C_f(D, C_t, 1 - C_t)

    RGB_c = L - D_c * L

    return RGB_c


class GamutMedicinaRGBSaturationWidget(GamutMedicinaBaseWidget):
    def __init__(self,
                 reference_image=None,
                 compression_threshold=0.8,
                 compression_function='tanh',
                 *args,
                 **kwargs):
        self._compression_threshold = compression_threshold
        self._compression_function = compression_function

        kwargs['enable_colour_wheel'] = False

        super().__init__(reference_image, *args, **kwargs)

        self._RGB_r = None

        if self._enable_chromaticity_diagram:
            self._RGB_d = None

        self._compression_threshold_chromaticity_diagram_plot = None

        self._reset_compression_controls_Button = None
        self._compression_threshold_FloatSlider = None

        self.initialize_data()
        self.initialize_axes()
        self.initialise_widgets()
        self.attach_callbacks()

        self.update_widget(None)

    def initialize_data(self):
        super().initialize_data()

        # *** Reference Image ***
        self._RGB_r = self._reference_image_working

        # *** Decimated Image, i.e. Scatter ***
        if self._enable_chromaticity_diagram:
            self._RGB_d = self._decimated_image_working

    def initialize_axes(self):
        super().initialize_axes()

    def initialise_widgets(self):
        super().initialise_widgets()

        # *** Widgets ***
        self._widget_name_HTML.value = (
            '<span style="color: #3498DB; font-size: 1.25rem">'
            '<b>RGB Saturation Widget</b></span>')

        self._reset_compression_controls_Button = (ipywidgets.widgets.Button(
            description="Reset Compression Controls"))
        compression_functions = list(COMPRESSION_FUNCTIONS.keys())
        self._compression_function_DropDown = ipywidgets.widgets.Dropdown(
            options=compression_functions,
            value=self._compression_function,
            description='Compression Function:',
        )
        self._compression_threshold_FloatSlider = (
            ipywidgets.widgets.FloatSlider(
                min=0.0,
                max=1.0,
                step=0.01,
                value=self._compression_threshold,
                description='Compression Threshold'))

        # *** Layout ***
        self._compression_controls_HBox = ipywidgets.widgets.HBox([
            self._reset_compression_controls_Button,
            self._compression_function_DropDown,
            self._compression_threshold_FloatSlider,
        ])

        self._controls_Tab.children += (self._compression_controls_HBox, )

        self._controls_Tab.set_title(2, 'Compression Controls')

    def attach_callbacks(self):
        super().attach_callbacks()

        self._reset_compression_controls_Button.on_click(
            self.reset_compression_controls)
        self._compression_function_DropDown.observe(self.update_widget,
                                                    'value')
        self._compression_threshold_FloatSlider.observe(
            self.update_widget, 'value')

    def reset_compression_controls(self, change):
        self._compression_threshold_FloatSlider.value = (
            self._compression_threshold)

        self.update_widget(None)

    def update_reference_image_axes(self, C_t, C_f, show_out_of_gamut):
        RGB_r = medicina_RGB_saturation(self._RGB_r, C_t, C_f)
        self._reference_image_mapped = RGB_r
        self._reference_image_imshow.set_data(
            self.working_space_to_display_space(
                RGB_r, show_out_of_gamut=show_out_of_gamut))

    def update_chromaticity_diagram_axes(self, C_t, C_f):
        RGB_r_s = medicina_RGB_saturation(self._RGB_d, C_t, C_f)
        self.update_scatter_plot(RGB_r_s)

    def update_widget(self, change):
        super().update_widget(change)

        if self._enable_medicina:
            EV = self._exposure_FloatSlider.value
            C_t = 1 - self._compression_threshold_FloatSlider.value
            C_f = COMPRESSION_FUNCTIONS[
                self._compression_function_DropDown.value]
            show_out_of_gamut = self._show_out_of_gamut_CheckBox.value
            if EV != self._exposure_value:
                self._exposure_value = EV

                self._RGB_r = self._reference_image_working = adjust_exposure(
                    self._reference_image_pre_working, EV)

                if self._enable_chromaticity_diagram:
                    self._RGB_d = self._decimated_image_working = adjust_exposure(
                        self._decimated_image_pre_working, EV)

            self.update_reference_image_axes(C_t, C_f, show_out_of_gamut)
            self.update_chromaticity_diagram_axes(C_t, C_f)

            self._figure.canvas.draw_idle()


GamutMedicinaRGBSaturationWidget(
    '../resources/images/A009C002_190210_R0EI_Alexa_LogCWideGamut.exr')

GamutMedicinaRGBSaturationWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding…

Note
----

> *Mansencal and Scharfenberg (2020)* gamut mapping study model twists the woodbard ceiling hues toward magenta in this example which is not pleasing but can be corrected with the Hue Controls. *Smith (2020)* gamut mapping study model is producing more appealing colours here.

In [19]:
GamutMedicinaHSVControlWidget(
    '../resources/images/A009C002_190210_R0EI_Alexa_LogCWideGamut.exr',
    enable_colour_wheel=False)

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…

In [20]:
GamutMedicinaRGBSaturationWidget(
    '../resources/images/Cornell_Box_Rigid_Spheres_190_Patch_Roughplastic_Point_Grey_Grasshopper_50S5C_RGB_W_A.exr')

GamutMedicinaRGBSaturationWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding…

In [21]:
GamutMedicinaRGBSaturationWidget(
    '../resources/images/5DMarkII_Spac-o-ween_001.exr')

GamutMedicinaRGBSaturationWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding…

In [22]:
GamutMedicinaRGBSaturationWidget(
    '../resources/images/Collage_01.exr')

GamutMedicinaRGBSaturationWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding…

## Compared Model Distortions

Various model distortions comparisons.

Note
----

> *Mansencal and Scharfenberg (2020)* gamut mapping study model tends to produce rounder surface area compared to that of *Smith (2020)*. Note also how the magenta area along the line of purples is much more compressed.

In [23]:
colour.write_image(
    colour_wheel(clip_circle=False)[..., 0:3],
    '../resources/images/Colour_Wheel.exr')

In [24]:
GamutMedicinaHSVControlWidget(
    '../resources/images/Colour_Wheel.exr',
    image_decimation=20,
    protected_area_threshold=0.5,
    compression_threshold=0.5,
    enable_colour_wheel=False)

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…

In [25]:
GamutMedicinaRGBSaturationWidget(
    '../resources/images/Colour_Wheel.exr',
    image_decimation=20,
    compression_threshold=0.5)

GamutMedicinaRGBSaturationWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding…

## Jhepc Extra Figures

In [26]:
GamutMedicinaHSVControlWidget(
    '../resources/images/5DMarkII_Spac-o-ween_001.exr',
    enable_colour_wheel=False,
    enable_medicina=False)

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…

In [27]:
GamutMedicinaHSVControlWidget(
    '../resources/images/5DMarkII_Spac-o-ween_001.exr',
    enable_colour_wheel=False,
    show_out_of_gamut=True,
    enable_medicina=False)

GamutMedicinaHSVControlWidget(children=(Output(layout=Layout(border='solid 4px #222', margin='4px', padding='0…